### Description

This changes the loss to incentivize boolean outputs:
1. Loss compares CNN(predicted_sequence) to CNN(actual_sequence) instead of CNN(predicted_sequence) to actual_expression.
2. Uses temperature-scaled softmax function to onehot encode outputs.

TODO:
1. Try either categorical cross-entropy at the output layer.
2. Use custom loss function to penalize deviating from 1 or 0.

In [1]:
import RNN_2_4 as parent

In [2]:
name = 'RNN_2_4'

In [3]:
file_path = '../Data/combined/LaFleur_supp.csv'

df, scaler = parent.load_and_preprocess_data(file_path)

In [4]:
X_sequence, X_expressions, y = parent.preprocess_X_y(df)

In [5]:
X_sequence_train, X_sequence_test, X_expressions_train, X_expressions_test, y_train, y_test = parent.train_test_split(
        X_sequence, X_expressions, y, test_size=0.2, random_state=42)

In [6]:
cnn_model = parent.load_model('../Models/CNN_5_0.keras')
lstm_model = parent.build_lstm_model(sequence_length=150, input_nucleotide_dim=5, output_nucleotide_dim=4, expression_dim=1)

In [7]:
loss_history = parent.train_model(lstm_model, cnn_model, X_sequence_train, X_expressions_train, y_train, batch_size=512, epochs=1, learning_rate=0.01)
lstm_model.save(f'../Models/{name}.keras')

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 2240, but received input with shape (512, 359808)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(512, 22500, 4), dtype=float32)
  • training=None
  • mask=None

In [8]:
lstm_model = parent.load_model(f'../Models/{name}.keras')

c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
mse, predicted_expression = parent.evaluate_model(lstm_model, cnn_model, X_sequence_test, X_expressions_test)
print(f'Mean Squared Error on Test Data: {mse:.4f}')

305/305 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Mean Squared Error on Test Data: 0.0170


In [10]:
parent.predict_with_lstm(lstm_model, 'TTTTCTATCTACGTACTTGACACTATTTCCTATTTCTCTTATAATT__________ACCTTAGTTTGTACGTT', 0.5, scaler, 150)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


c:\Users\rsore\anaconda3\envs\TX_prediction\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


'00000000000000000000000000000000000000000000000000000000000000000000000000000TTTTCTATCTACGTACTTGACACTATTTCCTATTTCTCTTATAATT0000000000ACCTTAGTTTGTACGTT'